In [1]:
import numpy as np
from sklearn.datasets import  make_blobs
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [14]:
scores = list()
for train_idx, test_idx in kfold.split(X):
    train_X, test_X = X[train_idx], X[test_idx]
    train_y, test_y = y[train_idx], y[test_idx]
    model = KNeighborsClassifier()
    model.fit(train_X, train_y)
    yhat = model.predict(test_X)
    acc = accuracy_score(test_y, yhat)
    scores.append(acc)
    print('> ',acc)
mean_s, std_s = np.mean(scores), np.std(scores)
print('Mean; %.3f, Standard Deviation: %.3f'%(mean_s, std_s))

>  0.936
>  0.956
>  0.924
>  0.94
>  0.936
>  0.924
>  0.93
>  0.936
>  0.942
>  0.944
>  0.962
>  0.95
>  0.95
>  0.952
>  0.94
>  0.946
>  0.94
>  0.95
>  0.95
>  0.952
Mean; 0.943, Standard Deviation: 0.010


In [34]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [51]:
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
X, y = make_blobs(n_samples = 12000, centers=2, n_features=100, cluster_std=20)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,stratify=y)

In [62]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)
tr_preds = model.predict(X_train)
preds = model.predict(X_test)
print(accuracy_score(y_train, tr_preds))
print(accuracy_score(y_test, preds))

0.9740740740740741
0.9525


In [64]:
oof_preds = np.zeros(X_train.shape[0])
sub_preds = np.zeros(y_test.shape[0])

kfold = KFold(n_splits=10, shuffle=True)

for n_fold, (train_idx, val_idx) in tqdm(enumerate(kfold.split(X_train,y_train))):
    train_X, val_X = X_train[train_idx], X_train[val_idx]
    train_y, val_y = y_train[train_idx], y_train[val_idx]
    
    model = KNeighborsClassifier()
    model.fit(train_X, train_y)
    
    oof_preds[val_idx] = model.predict(val_X)
    test_preds = model.predict(X_test)
    sub_preds += test_preds / kfold.n_splits
print(accuracy_score(y_train, oof_preds))
sub_preds = (sub_preds>=0.5).astype(int)
print(accuracy_score(y_test, sub_preds))
oof_1 = oof_preds

10it [01:00,  6.03s/it]

0.9519444444444445
0.9525


In [65]:
from sklearn.model_selection import StratifiedKFold
oof_preds = np.zeros(X_train.shape[0])
sub_preds = np.zeros(y_test.shape[0])

skfold = StratifiedKFold(n_splits=10, shuffle=True)

for n_fold, (train_idx, val_idx) in tqdm(enumerate(skfold.split(X_train,y_train))):
    train_X, val_X = X_train[train_idx], X_train[val_idx]
    train_y, val_y = y_train[train_idx], y_train[val_idx]
    
    model = KNeighborsClassifier()
    model.fit(train_X, train_y)
    
    oof_preds[val_idx] = model.predict(val_X)
    test_preds = model.predict(X_test)
    sub_preds += test_preds / kfold.n_splits
print(accuracy_score(y_train, oof_preds))
sub_preds = (sub_preds>=0.5).astype(int)
print(accuracy_score(y_test, sub_preds))
oof_2 = oof_preds

10it [01:01,  6.12s/it]

0.9518518518518518
0.9525


### Voting Classifier

In [77]:
import statistics

In [79]:
preds = []

for x, y, z in (zip(tr_preds, oof_1, oof_2)):
    value = statistics.mode([x,y,z])
    preds.append(value)

In [81]:
accuracy_score(y_train, preds)

0.9575